In [67]:
import pandas as pd
import os
import mysql.connector
from mysql.connector import Error
from dotenv import load_dotenv

In [68]:
base_dir = os.getcwd()

### import csv to mysql (users, vehicles, transactions)

In [69]:
#  Read csv files
users = pd.read_csv(f"{base_dir}\\data\\users.csv")
vehicles = pd.read_csv(f"{base_dir}\\data\\vehicles.csv")
transactions = pd.read_csv(f"{base_dir}\\data\\rental_transactions.csv")

In [70]:
def generate_insert_into(df, table_name="A"):
    columns = df.columns.tolist()
    columns_str = ", ".join(columns)
    placeholders = ", ".join(["%s"] * len(columns))
    insert_stmt = f"INSERT INTO {table_name} ({columns_str}) VALUES ({placeholders});"
    return insert_stmt

In [71]:
# query create table for users, vehicles, and transaction
query_create_table_users = """
CREATE TABLE users (
    user_id VARCHAR(50),
    first_name VARCHAR(255),
    last_name VARCHAR(255),
    email VARCHAR(255),
    phone_number VARCHAR(255),
    driver_license_number VARCHAR(50),
    driver_license_expiry DATE,
    creation_date DATE,
    is_active TINYINT(1)
);
"""

query_create_table_vehicles = """
CREATE TABLE vehicles (
    active TINYINT(1),
    vehicle_license_number VARCHAR(255),
    registration_name TEXT,
    license_type VARCHAR(255),
    expiration_date VARCHAR(50),
    permit_license_number VARCHAR(255),
    certification_date DATE,
    vehicle_year YEAR,
    base_telephone_number VARCHAR(255),
    base_address TEXT,
    vehicle_id VARCHAR(50),
    last_update_timestamp VARCHAR(50),
    brand VARCHAR(255),
    vehicle_type VARCHAR(255)
);
"""

query_create_table_transactions = """
CREATE TABLE transactions (
    rental_id VARCHAR(50),
    user_id VARCHAR(50),
    vehicle_id VARCHAR(50),
    rental_start_time DATETIME,
    rental_end_time DATETIME,
    pickup_location VARCHAR(255),
    dropoff_location VARCHAR(255),
    total_amount FLOAT
);
"""


In [72]:
# query insert table
query_insert_table_users = generate_insert_into(users, 'users')
query_insert_table_vehicle = generate_insert_into(vehicles, 'vehicles')
query_insert_table_transaction = generate_insert_into(transactions, 'transactions')

In [77]:
# Load .env
load_dotenv()

# Read MySQL connection info
hostname = os.getenv("MYSQL_HOST")
port = int(os.getenv("MYSQL_PORT"))
username = os.getenv("MYSQL_USER")
password = os.getenv("MYSQL_PASSWORD")

try:
    conn = mysql.connector.connect(
        host=hostname,
        user=username,
        password=password,
        port=port
    )
    
    if conn.is_connected():
        print("[INFO] connect to mysql success")
        cursor = conn.cursor()
        cursor.execute("CREATE DATABASE IF NOT EXISTS rental_vehicle;")
        cursor.execute("USE rental_vehicle;")
        cursor.execute("SELECT DATABASE();")
        record = cursor.fetchone()
        print(f"[INFO] you're connected to {record[0]} database")

        cursor.execute("DROP TABLE IF EXISTS users;")
        print("[INFO] drop table users if exist success")
        cursor.execute("DROP TABLE IF EXISTS vehicles;")
        print("[INFO] drop table vehicles if exist success")
        cursor.execute("DROP TABLE IF EXISTS transactions;")
        print("[INFO] drop table transactions if exist success")

        cursor.execute(query_create_table_users)
        print("[INFO] Create table user success")
        cursor.execute(query_create_table_vehicles)
        print("[INFO] Create table vehicles success")
        cursor.execute(query_create_table_transactions)
        print("[INFO] Create table transactions success")

        batch_size = 5000
        tables = {
            'users': users,
            'vehicles': vehicles,
            'transactions': transactions
        }
        insert_query = {
            'users': query_insert_table_users,
            'vehicles': query_insert_table_vehicle,
            'transactions': query_insert_table_transaction
        }
        for tbl in tables.keys():
            num_batches = len(tables[tbl]) // batch_size + 1
            for i in range(num_batches):
                start_idx = i * batch_size
                end_idx = ((i+1) * batch_size)
                batch_data = tables[tbl].iloc[start_idx:end_idx]
                batch_data_record = [tuple(row) for row in batch_data.to_numpy()]

                cursor.executemany(insert_query[tbl], batch_data_record)
                conn.commit()
                print(f"[INFO] insert batch {i+1}/{num_batches} success")

            cursor.execute(f"SELECT COUNT(*) FROM {tbl};")
            count_data = cursor.fetchone()
            print(f"[INFO] all {count_data[0]} {tbl} data inserted successfully")

except Error as e:
    print("Error while connecting to MySQL", e)

finally:
    if conn.is_connected():
        cursor.close()
        conn.close()
        print("MySQL connection is closed")


[INFO] connect to mysql success
[INFO] you're connected to rental_vehicle database
[INFO] drop table users if exist success
[INFO] drop table vehicles if exist success
[INFO] drop table transactions if exist success
[INFO] Create table user success
[INFO] Create table vehicles success
[INFO] Create table transactions success
[INFO] insert batch 1/7 success
[INFO] insert batch 2/7 success
[INFO] insert batch 3/7 success
[INFO] insert batch 4/7 success
[INFO] insert batch 5/7 success
[INFO] insert batch 6/7 success
[INFO] insert batch 7/7 success
[INFO] all 30000 users data inserted successfully
[INFO] insert batch 1/22 success
[INFO] insert batch 2/22 success
[INFO] insert batch 3/22 success
[INFO] insert batch 4/22 success
[INFO] insert batch 5/22 success
[INFO] insert batch 6/22 success
[INFO] insert batch 7/22 success
[INFO] insert batch 8/22 success
[INFO] insert batch 9/22 success
[INFO] insert batch 10/22 success
[INFO] insert batch 11/22 success
[INFO] insert batch 12/22 success


In [74]:
len(users), len(vehicles), len(transactions)

(30000, 109584, 20080)

### cleaning location files

In [78]:
locs = pd.read_csv(f"{base_dir}\\data\\locations.csv")

In [79]:
locs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   location_id    300 non-null    int64  
 1   location_name  300 non-null    object 
 2   address        300 non-null    object 
 3   city           300 non-null    object 
 4   state          300 non-null    object 
 5   zip_code       300 non-null    int64  
 6   latitude       300 non-null    float64
 7   longitude      300 non-null    float64
dtypes: float64(2), int64(2), object(4)
memory usage: 18.9+ KB


In [80]:
locs.head(3)

,location_id,location_name,address,city,state,zip_code,latitude,longitude
0,2702,"Jackson, Velazquez and Gonzales",3140 Heath Radial Apt. 604,Modesto,CA,94540,86.258020,-169.244800
1,4380,Bean LLC,51144 Patrick Isle Suite 397,Fontana,CA,92188,-74.455893,-42.279882
2,7709,Gilbert-Simmons,4738 Lewis Locks,Roseville,CA,91032,-65.430931,-64.763489


In [82]:
locs.isnull().sum()

location_id      0
location_name    0
address          0
city             0
state            0
zip_code         0
latitude         0
longitude        0
dtype: int64

In [85]:
locs.duplicated(subset=['location_id']).sum()

np.int64(0)

location.csv is clean. 